[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[TensorFlow Speech Recognition Challenge](https://www.kaggle.com/c/tensorflow-speech-recognition-challenge)

[WavCeption V1: a 1-D Inception approach (LB 0.76)](https://www.kaggle.com/code/ivallesp/wavception-v1-a-1-d-inception-approach-lb-0-76)

# WavCeption V1: just a 1-D Inception approach

I just wanted to share a little toy I have been playing with and gave me **amazing results**. As I currently don't have time, I would like to share it to see how people plays with it :-D. **The WavCeption V1** network seems to produce impressive results compared to a regular convolutional neural network, but in this competition it seems that there is a hard-work on the pre-processing and unknown tracks management. It is based on the Google's inception network, the same idea.

I wrote some weeks ago a module implementing it so that it is easy to build an 1D-inception network by connecting lots of these modules in cascade (as you will see below).

Unfortunately and due to several Kaggle constraints, it won't run in the kernel machine, so I encourage you to download it and run it in your own machine.

By running the model for 12h without struggling too much I achieved 0.76 in the leaderboard (with 0.84 in local test). Some other trials in the same line gave me 0.89 in local, so there is a huge improvement in how you deal with the unknown clips :-D

# DeepL 번역
제가 가지고 놀다가 **놀라운 결과**를 얻은 작은 장난감을 공유하고 싶었습니다. 지금은 시간이 없어서 사람들이 어떻게 가지고 노는지 보기 위해 공유하고 싶습니다 :-D. **WavCeption V1** 네트워크는 일반 컨볼루션 신경망에 비해 인상적인 결과를 내는 것 같은데, 이번 대회에서는 전처리와 미지의 트랙 관리에 공을 들인 것 같습니다. 구글의 인셉션 네트워크와 같은 아이디어를 기반으로 하고 있습니다.

저는 몇 주 전에 이를 구현하는 모듈을 작성했는데, 아래에서 보시는 것처럼 이 모듈을 많이 계단식으로 연결하면 1D-인셉션 네트워크를 쉽게 구축할 수 있습니다.

안타깝게도 몇 가지 Kaggle 제약 조건으로 인해 커널 머신에서는 실행되지 않으므로 직접 다운로드하여 자신의 머신에서 실행해 보시기 바랍니다.

큰 어려움 없이 12시간 동안 모델을 실행하여 리더보드에서 0.76을 달성했습니다(로컬 테스트에서는 0.84). 같은 라인의 다른 테스트에서는 로컬에서 0.89를 기록했으니, 알 수 없는 클립을 처리하는 방법이 크게 개선되었습니다 :-D

## Load modules and libraries

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
# %matplotlib inline
get_ipython().run_line_magic('matplotlib', 'inline')    # 매직명렁어 안 될 때
import numpy as np
import pandas as pd
import os
import shutil
import glob
import random
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import IPython
from numpy.fft import rfft, irfft
# import numpy as np
# import  random
import itertools

from scipy.io import wavfile
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy as sp
import  tensorflow as tf

2025-03-04 12:48:29.910236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741060110.210057   14877 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741060110.298915   14877 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 12:48:31.114203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Noise generation functions

The code in this section has been borrowed and adapted from:  
https://github.com/python-acoustics/python-acoustics/blob/master/acoustics/generator.py

In [3]:
def ms(x):
    """Mean value of signal 'x' squared.
    :param x: Dynamic quantity.
    :returns: Mean squared of 'x'.
    """
    return (np.abs(x)**2.0).mean()

def normalize(y, x=None):
    """normalize power in y to a (standard normal) white noise signal.
    Optionally normalize to power in signal `x`.
    # The mean power of a Gaussian with :math: `\\mu=0` and :math:`\\sigma=1` is 1.
    """
    # return y * np.sqrt( (np.abs(x)**2.0).mean() / (np.abs(y)**2.0).mean() )
    if x is not None:
        x = ms(x)
    else:
        x = 1.0
    return y * np.sqrt( x / ms(y) )
    # return  y * np.sqrt( 1.0 / np.abs(y)**2.0).mean() )

https://www.kaggle.com/code/ivallesp/wavception-v1-a-1-d-inception-approach-lb-0-76